# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('/Users/malamapono/JupyterProjects/AlgorithmicTrading/sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'companyName': 'Apple Inc', 'marketcap': 3031373626809, 'week52high': 189.45, 'week52low': 119.16, 'week52highSplitAdjustOnly': 184.41, 'week52lowSplitAdjustOnly': 121.17, 'week52change': 0.346532707195549, 'sharesOutstanding': 16739666089, 'float': 0, 'avg10Volume': 69383533, 'avg30Volume': 108714482, 'day200MovingAvg': 157.72, 'day50MovingAvg': 169.53, 'employees': 151998, 'ttmEPS': 11.33, 'ttmDividendRate': 0.8640898079668944, 'dividendYield': 0.005084826633960812, 'nextDividendDate': '', 'exDividendDate': '2021-10-29', 'nextEarningsDate': '2022-01-16', 'peRatio': 15.48211721032954, 'beta': 1.342427086499515, 'maxChangePercent': 70.2971755103955, 'year5ChangePercent': 5.783948448413933, 'year2ChangePercent': 1.5117580135956976, 'year1ChangePercent': 0.3612742081360891, 'ytdChangePercent': 0, 'month6ChangePercent': 0.3105678359033371, 'month3ChangePercent': 0.25649081747394326, 'month1ChangePercent': 0.07918918514903722, 'day30ChangePercent': 0.0868760300293088, 'day5ChangePercent':

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.3612742081360891

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_df = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        one_year = data[symbol]['stats']['year1ChangePercent']
        price = data[symbol]['price']
        series = pd.Series([symbol,price,one_year,'NA'],index=my_columns)
        final_df = final_df.append(series,ignore_index=True)


In [7]:
final_df.head()

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,166.88,0.359862,NA
1,AAL,18.65,0.143627,NA
2,AAP,244.17,0.54797,NA
3,AAPL,185.94,0.353268,NA
4,ABBV,135.90,0.326746,NA


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
final_df['One-Year Price Return'] = final_df['One-Year Price Return'].astype('float64')
final_df = final_df.nlargest(50,'One-Year Price Return')
final_df.reset_index(inplace=True)
final_df.drop('index',axis=1,inplace=True)
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,80.11,2.378889,NA
1,DVN,45.14,1.984903,NA
2,MCHP,90.56,1.614318,NA
3,MRO,16.69,1.519506,NA
4,FTNT,372.90,1.444212,NA
5,F,21.64,1.377630,NA
6,NVDA,299.00,1.290442,NA
7,FANG,112.63,1.277593,NA
8,NUE,116.72,1.225734,NA
9,IT,343.44,1.089510,NA


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
investment_size = float(input('What is your investment size: '))
position_size = investment_size/50
final_df['Number of Shares to Buy'] = position_size/final_df['Price']

What is your investment size: 1000000


In [10]:
final_df.head()

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,80.11,2.378889,249.656722
1,DVN,45.14,1.984903,443.066017
2,MCHP,90.56,1.614318,220.848057
3,MRO,16.69,1.519506,1198.322349
4,FTNT,372.90,1.444212,53.633682


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [11]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_df = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        one_year = data[symbol]['stats']['year1ChangePercent']
        six_month = data[symbol]['stats']['month6ChangePercent']
        three_month = data[symbol]['stats']['month3ChangePercent']
        one_month = data[symbol]['stats']['month1ChangePercent']
        price = data[symbol]['price']
        series = pd.Series([symbol,price,'NA',one_year,'NA',six_month,'NA',three_month,'NA',one_month,'NA','NA'],index=hqm_columns)
        hqm_df = hqm_df.append(series,ignore_index=True)
        
hqm_df.head()        

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,164.69,NA,0.359309,NA,0.083961,NA,0.025671,NA,0.07993,NA,NA
1,AAL,18.41,NA,0.145201,NA,-0.165771,NA,-0.173694,NA,0.103863,NA,NA
2,AAP,249.96,NA,0.569209,NA,0.165601,NA,0.160522,NA,0.104147,NA,NA
3,AAPL,178.61,NA,0.346717,NA,0.30996,NA,0.251439,NA,0.077864,NA,NA
4,ABBV,140.90,NA,0.328619,NA,0.217444,NA,0.258843,NA,0.170443,NA,NA


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [12]:
# There are some none values in the hqm_df as the data was not
# properly returned or simply does not exist in api anymore
# We need to remove these nan values from the hqm_df before
# we calculate percentile scores. 
is_NaN = hqm_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
hqm_df[row_has_NaN]

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
118,CTL,11.000,NA,None,NA,None,NA,None,NA,None,NA,NA
165,ETFC,51.260,NA,None,NA,None,NA,None,NA,None,NA,NA
326,MYL,16.071,NA,None,NA,None,NA,None,NA,None,NA,NA
327,NBL,8.820,NA,None,NA,None,NA,None,NA,None,NA,NA


In [13]:
# Dont know why this implementation doesn't work with 
# negatives values in price return columns
time_periods = ['One-Year','Six-Month','Three-Month','One-Month']
for time_period in time_periods:
    column = f'{time_period} Price Return'
    hqm_df = hqm_df.dropna(axis=0)
    hqm_df[f'{time_period} Return Percentile'] = hqm_df[column].apply(lambda x:score(hqm_df[column],x)/100)

hqm_df.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,164.69,NA,0.359309,0.618762,0.083961,0.512974,0.025671,0.357285,0.07993,0.467066,NA
1,AAL,18.41,NA,0.145201,0.319361,-0.165771,0.073852,-0.173694,0.025948,0.103863,0.664671,NA
2,AAP,249.96,NA,0.569209,0.846307,0.165601,0.708583,0.160522,0.726547,0.104147,0.668663,NA
3,AAPL,178.61,NA,0.346717,0.606786,0.30996,0.924152,0.251439,0.912176,0.077864,0.459082,NA
4,ABBV,140.90,NA,0.328619,0.582834,0.217444,0.812375,0.258843,0.926148,0.170443,0.942116,NA


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [14]:
hqm_df['HQM Score'] = hqm_df[['One-Year Return Percentile','Six-Month Return Percentile','Three-Month Return Percentile','One-Month Return Percentile']].mean(axis=1)
hqm_df.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,164.69,NA,0.359309,0.618762,0.083961,0.512974,0.025671,0.357285,0.07993,0.467066,0.489022
1,AAL,18.41,NA,0.145201,0.319361,-0.165771,0.073852,-0.173694,0.025948,0.103863,0.664671,0.270958
2,AAP,249.96,NA,0.569209,0.846307,0.165601,0.708583,0.160522,0.726547,0.104147,0.668663,0.737525
3,AAPL,178.61,NA,0.346717,0.606786,0.30996,0.924152,0.251439,0.912176,0.077864,0.459082,0.725549
4,ABBV,140.90,NA,0.328619,0.582834,0.217444,0.812375,0.258843,0.926148,0.170443,0.942116,0.815868


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [15]:
hqm_df = hqm_df.nlargest(50,'HQM Score')
hqm_df.reset_index(inplace=True)
hqm_df.drop('index',axis=1,inplace=True)
hqm_df.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ANET,144.93,NA,0.994292,0.976048,0.587951,0.996008,0.659383,1.000000,0.192343,0.968064,0.985030
1,EXR,230.94,NA,1.017888,0.980040,0.409401,0.974052,0.360311,0.978044,0.147091,0.880240,0.953094
2,AVGO,690.60,NA,0.583648,0.856287,0.442694,0.980040,0.382052,0.982036,0.215857,0.988024,0.951597
3,FTNT,367.70,NA,1.470155,0.992016,0.487968,0.986028,0.20873,0.840319,0.185507,0.964072,0.945609
4,AZO,2116.84,NA,0.802579,0.958084,0.383948,0.960080,0.255177,0.922156,0.164858,0.920160,0.940120


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [16]:
hqm_df['Number of Shares to Buy'] = position_size/hqm_df['Price']
hqm_df.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ANET,144.93,137.997654,0.994292,0.976048,0.587951,0.996008,0.659383,1.000000,0.192343,0.968064,0.985030
1,EXR,230.94,86.602581,1.017888,0.980040,0.409401,0.974052,0.360311,0.978044,0.147091,0.880240,0.953094
2,AVGO,690.60,28.960324,0.583648,0.856287,0.442694,0.980040,0.382052,0.982036,0.215857,0.988024,0.951597
3,FTNT,367.70,54.392168,1.470155,0.992016,0.487968,0.986028,0.20873,0.840319,0.185507,0.964072,0.945609
4,AZO,2116.84,9.448045,0.802579,0.958084,0.383948,0.960080,0.255177,0.922156,0.164858,0.920160,0.940120


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [17]:
writer = pd.ExcelWriter('momentum_strategy.xlsx',engine='xlsxwriter')
hqm_df.to_excel(writer,sheet_name='Momentum Strategy',index=False)


## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [18]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.000000',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [19]:
column_formats = {
                'A':['Ticker',string_template],
                'B':['Price',dollar_template ],
                'C':['Number of Shares to Buy',float_template],
                'D':['One-Year Price Return',percent_template],
                'E':['One-Year Return Percentile',percent_template],
                'F':['Six-Month Price Return',percent_template],
                'G':['Six-Month Return Percentile',percent_template],
                'H':['Three-Month Price Return',percent_template],
                'I':['Three-Month Return Percentile',percent_template],
                'J':['One-Month Price Return',percent_template],
                'K':['One-Month Return Percentile',percent_template],
                'L':['HQM Score',percent_template]
}

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [20]:
for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].write(f'{column}1',column_formats[column][0],string_template)
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}',25,column_formats[column][1]) 

In [21]:
writer.save()